In [1]:
import math
import os
import random
import time
from numpy import interp
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
from gtda.diagrams import Scaler, PersistenceEntropy, BettiCurve, PersistenceLandscape, Silhouette
from gtda.homology import VietorisRipsPersistence
from gtda.pipeline import make_pipeline
from gtda.time_series import SlidingWindow, TakensEmbedding
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
def label_prefog(dataset,window_length = 1):#一般步长都是一到4
  dataset.drop(index = list(dataset[dataset['Action'] == 0].index),inplace=True)#axis删除行，默认为0,删除列默认为1，index直接指定要删除的行，inplace为真则删除数据不返回，为假则返回删除后的表
  window_length = 64*window_length
    
  fog_index=[]
  for i in dataset.index:
      if dataset.loc[i,'Action'] == 2:#取Action列的第i行的数据等于2
        fog_index.append(i)
  fog_index



  start_indices=[]
  for i in fog_index:
    if (dataset.loc[i-1,'Action']!=dataset.loc[i,'Action']):
      start_indices.append(i)
 

  prefog=[]
  for start in start_indices:
    prefog_start = [x for x in range(start-window_length,start)]#列表解析式一个嵌套循环
    prefog.append(prefog_start)

  prefog = [item for sublist in prefog for item in sublist]#两个循环，sublist（子表）在prefog中，item在sublist中，输出item

  for i in prefog:
       dataset.loc[i,'Action'] = 3
  dataset['Action'] = dataset['Action'] - 1
  return dataset

In [7]:
import os
import pandas as pd
data_path = './dataset/'

people = []
dataset = pd.DataFrame()
for person in os.listdir(data_path):#返回指定路径下的文件夹
    if '.txt' in person: 
        people.append(person)
        
window_length = 2
for person in people: 
    name = person.split('R')[0]#分割R次并取序列为0的项
    print (name)
    file = data_path+'/'+person
    temp = pd.read_csv(file,delimiter= " ", header = None)
    print (person,' is read',end = '\t')
    if 2 in temp[max(temp.columns)].unique():
        print ('Adding {} to dataset'.format(person),end = '\t')#将人加入到dataset中
        temp.columns = ['time','A_F','A_V','A_L','L_F','L_V','L_L','T_F','T_V','T_L','Action']    
        temp = label_prefog(temp,window_length).reset_index(drop=True)
        temp['name'] = name
        print ('{} is labelled'.format(person))#不指定位置，打印人名
        dataset = pd.concat([dataset,temp],axis = 0)#数据的拼接，在axis在行上拼接，=1在列上拼接

    print ('')
dataset.reset_index(drop =True,inplace=True) #重新排列顺序
to_path = './'
to_name = to_path +'win_'+str(window_length)+".csv"

dataset.to_csv(to_name,index = False)

S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled

S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled

S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled

S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled

S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled

S03
S03R03.txt  is read	
S04
S04R01.txt  is read	
S05
S05R01.txt  is read	Adding S05R01.txt to dataset	S05R01.txt is labelled

S05
S05R02.txt  is read	Adding S05R02.txt to dataset	S05R02.txt is labelled

S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled

S06
S06R02.txt  is read	
S07
S07R01.txt  is read	Adding S07R01.txt to dataset	S07R01.txt is labelled

S07
S07R02.txt  is read	Adding S07R02.txt to dataset	S07R02.txt is labelled

S08
S08R01.txt  is read	Adding S08R01.txt to dataset	S08R01.txt is labelled

S

In [8]:
def create_window(act, window_length, dataframe):
    indices = list(dataframe[dataframe.Action == act].index)
    groups = []
    temp = []
    group_count = 0
    for i in range(len(indices)):
        if i == len(indices) - 1:
            temp.append(indices[i])
            groups.append(temp)
            temp = []
            break
        temp.append(indices[i])
        if indices[i] + 1 != indices[i + 1]:
            group_count += 1
            groups.append(temp)
            temp = []

    fs = 64

    final_dataframe = pd.DataFrame()
    for i in groups:
        required = math.floor(len(i) / (window_length * fs))

        req_index = i[0:(required * window_length * fs)]

        final_dataframe = pd.concat([final_dataframe, dataframe.iloc[req_index, :]], axis=0)
    return final_dataframe


In [9]:

def sbj_df(df, sbj='S01'):
    DF0 = create_window(0, 2, df)
    DF0 = DF0[DF0['name'] == sbj]
    DF1 = create_window(1, 2, df)
    DF1 = DF1[DF1['name'] == sbj]
    DF2 = create_window(2, 2, df)
    DF2 = DF2[DF2['name'] == sbj]
    return DF0, DF1, DF2


def gtda(dataframe, w=128):
    all_data = []
    dataframe = dataframe.drop(columns=['time', 'Action', 'name'])
    for i in range(0, len(dataframe), w):
        data = dataframe.iloc[i:i + w]
        data = data.to_numpy().transpose()
        if data.shape[1] == w:
            all_data.append(data)
    all_data = np.array(all_data)
    steps = [TakensEmbedding(time_delay=5, dimension=3),
             VietorisRipsPersistence(),
             Scaler()
             ]
    tda_pipe = make_pipeline(*steps)
    diagrams = tda_pipe.fit_transform(all_data)
    BC = BettiCurve(n_bins=50).fit_transform(diagrams)
    PL = PersistenceLandscape(n_bins=50).fit_transform(diagrams)
    SL = Silhouette(n_bins=50).fit_transform(diagrams)

    return np.mean(BC, axis=1), np.mean(PL, axis=1), np.mean(SL, axis=1)


def training(x, y, sbj_name):
    y = np.array(y)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)
    model = RandomForestClassifier(random_state=1)
    model.fit(x_train, y_train)
    y_pre = model.predict(x_test)
    print(classification_report(y_test, y_pre, digits=4))
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_pre)
    roc_auc = metrics.auc(fpr, tpr)
    print('AUC:', roc_auc)
    print('\n')



In [10]:

def get_random_list(start, stop, n):
    '''
    生成范围在[start,stop], 长度为n的数组.
    区间包含左右endpoint
    '''
    arr = list(range(start, stop + 1))
    shuffle_n(arr, n)
    return arr[-n:]


def shuffle_n(arr, n):
    random.seed(time.time())
    for i in range(len(arr) - 1, len(arr) - n - 1, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]

In [11]:
subject_list = ['S01', 'S02', 'S03', 'S05', 'S06', 'S07', 'S08', 'S09']
df = pd.read_csv('./win_2.csv')
for sbj in subject_list:
    DF0, DF1, DF2 = sbj_df(df, sbj=sbj)
    NOR_DF = pd.concat([DF0, DF2])
    nor_bc, nor_pl, nor_sl = gtda(NOR_DF)
    fog_bc, fog_pl, fog_sl = gtda(DF1)

    fog = [fog_bc, fog_pl, fog_sl]
    normal = [nor_bc, nor_pl, nor_sl]
    print(sbj)

    for length, nor_feature in enumerate(normal):
        print('{length} feature:')
        fog_feature = fog[length]
        idx = get_random_list(0, len(nor_feature) - 1, int(len(fog_feature) * 1))
        nor_feature = nor_feature[idx]

        X = np.concatenate([nor_feature, fog_feature], axis=0)
        Y = []
        Y += len(nor_feature) * [0]
        Y += len(fog_feature) * [1]

        # x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=True)
        training(X, Y, sbj)

    all_fog = np.concatenate(fog, axis=1)
    all_nor = np.concatenate(normal, axis=1)
    all_X = np.concatenate([all_nor, all_fog], axis=0)
    all_Y = []
    all_Y += len(all_nor) * [0]
    all_Y += len(all_fog) * [1]
    print('feature union')
    training(all_X, all_Y, sbj)


S01
{length} feature:
              precision    recall  f1-score   support

           0     1.0000    0.7857    0.8800        14
           1     0.7692    1.0000    0.8696        10

    accuracy                         0.8750        24
   macro avg     0.8846    0.8929    0.8748        24
weighted avg     0.9038    0.8750    0.8757        24

AUC: 0.8928571428571428


{length} feature:
              precision    recall  f1-score   support

           0     0.9000    0.8182    0.8571        11
           1     0.8571    0.9231    0.8889        13

    accuracy                         0.8750        24
   macro avg     0.8786    0.8706    0.8730        24
weighted avg     0.8768    0.8750    0.8743        24

AUC: 0.8706293706293706


{length} feature:
              precision    recall  f1-score   support

           0     1.0000    0.8824    0.9375        17
           1     0.7778    1.0000    0.8750         7

    accuracy                         0.9167        24
   macro avg     0

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       225
           1     1.0000    1.0000    1.0000        10

    accuracy                         1.0000       235
   macro avg     1.0000    1.0000    1.0000       235
weighted avg     1.0000    1.0000    1.0000       235

AUC: 1.0


S08
{length} feature:
              precision    recall  f1-score   support

           0     0.7879    0.7879    0.7879        33
           1     0.7083    0.7083    0.7083        24

    accuracy                         0.7544        57
   macro avg     0.7481    0.7481    0.7481        57
weighted avg     0.7544    0.7544    0.7544        57

AUC: 0.7481060606060607


{length} feature:
              precision    recall  f1-score   support

           0     0.6774    0.7500    0.7119        28
           1     0.7308    0.6552    0.6909        29

    accuracy                         0.7018        57
   macro avg     0.7041    0.7026    0.7014        